# M53 - MGE

Oligopoly with free entry.

|Markets  |   X   |     N  |      Y  |      W    |  CONS |  ENTRE |
|---------|-------|--------|---------|-----------|-------|------- |
|    PX   | 100   |        |         |   -100    |       |        |  
|    PY   |       |        |    100  |   -100    |       |        |  
|    PF   |       |    20  |         |           |       |    -20 | 
|    PU   |       |        |         |    200    |  -200 |        |
|    PW   | -32   |    -8  |    -60  |           |   100 |        | 
|    PZ   | -48   |   -12  |    -40  |           |   100 |        | 
|    MK   | -20   |        |         |           |       |     20 |  

In [1]:
using MPSGE

In [5]:
M53 = MPSGEModel()

@parameters(M53, begin
    σ, 1
    ENDOW, 1
end)

@sectors(M53, begin
	X,	(description = "Activity level - sector X output",)
	Y,	(description = "Activity level - competitive sector Y",)
	W,	(description = "Welfare index for the representative consumer",)
	N,	(description = "Activity level - sector X fixed costs = no. of firms",)
end)
@commodities(M53, begin
	PU,	(description = "Price index for representative agent utility",)
	PX,	(description = "Price of good X (gross of markup)",)
	PY,	(description = "Price of good Y",)
	PF,	(description = "Unit price of inputs to fixed cost",)
	PW,	(description = "Price index for labor",)
	PZ,	(description = "Price index for capital",)
end)
@consumers(M53, begin
	CONS,	(description = "Representative agent",)
	ENTRE,	(description = "Entrepreneur (converts markup revenue to fixed cost)",)
end)

@auxiliaries(M53, begin
	MARKUP,	(description = "Optimal markup based on Marshallian demand elasticity",)
end)


@production(M53, X, [s=1,t=0], begin
    @output(PX, 80, t, taxes = [Tax(ENTRE, MARKUP)])
    @input(PW, 32, s)
    @input(PZ, 48, s)
end)

@production(M53, Y, [s=1,t=0], begin
    @output(PY, 100, t)
    @input(PW, 60, s)
    @input(PZ, 40, s)
end)

@production(M53, N, [s=1,t=0], begin
    @output(PF, 20/5, t)
    @input(PW, 8/5, s)
    @input(PZ, 12/5, s)
end)

@production(M53, W, [s=1,t=0], begin
    @output(PU, 200, t)
    @input(PX, 80, s, reference_price = 1.25)
    @input(PY, 100, s)
end)

@demand(M53, CONS, begin
    @final_demand(PU, 200)
end,begin
    @endowment(PW, 100*ENDOW)
    @endowment(PZ, 100*ENDOW)
end)


@demand(M53, ENTRE, begin
    @final_demand(PF, 20)
end,begin
end)

@aux_constraint(M53, MARKUP,
    MARKUP*N - 1
);

# Benchmark

In [6]:
set_start_value(N, 5)
set_start_value(PX, 1.25)
set_start_value(MARKUP, .2)

solve!(M53, cumulative_iteration_limit=0)
generate_report(M53)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_46F7.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 12

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (zero_profit[X)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.063000
Residual. . . . . . . . 0.000000e+00
Postsolved residual: 0.0000e+00


Solver Status: LOCALLY_SOLVED
Model Status: FEASIBLE_POINT

Default price normalization using income for CONS - This value is fixed. Unfix with unfix(CONS).

Row,var,value,margin
,GenericV…,Float64,Float64
1,Y,1.0,0.0
2,W,1.0,0.0
3,X,1.0,0.0
4,N,5.0,0.0
5,PF,1.0,0.0
6,PU,1.0,0.0
7,PW,1.0,0.0
8,PZ,1.0,0.0
9,PY,1.0,0.0


# Counterfactual

In [8]:
set_value!(ENDOW, 2)
fix(CONS,400)

solve!(M53)
generate_report(M53)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_2EC8.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 12

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             2.8144e+02             0.0e+00 (market_clearance[PW)
    1     1     0    12 1.5551e+01  1.0e+00    0.0e+00 (market_clearance[PY)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 1.5551e+01           I 0.0e+00 1.2e+01 (market_clearanc)
    1     1     3     3 3.5483e-01  1.0e+00 SO 0.0e+00 3.0e-01 (income_balance[)
    2     1     4     4 1.6763e-03  1.0e+00 SO 0.0e+00 1.1e-03 (market_clearanc)
    3     1     5     5 1.1899e-08  1.0e+00 SO 0.0e+00 1.1e-08 (income_balance[)

Major Iterations. . . . 3
Minor Iteration

Row,var,value,margin
,GenericV…,Float64,Float64
1,Y,2.0,-2.16005e-12
2,W,2.07193,-3.28328e-10
3,X,2.14645,1.56377e-9
4,N,7.07107,-6.17284e-14
5,PF,1.0,1.31295e-9
6,PU,0.965284,8.78799e-10
7,PW,1.0,-3.86535e-11
8,PZ,1.0,4.72085e-11
9,PY,1.0,-1.52139e-9
